# 修正RHEA中的反应
> zhenkun.shi@tib.cas.cn   
> 2024-09-23

In [1]:
import sys,os
sys.path.insert(0, os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(1,'../../')
from config import conf as cfg
from tools import filetool as ftool
from tools import bioFunctionLib as bfl
from tools import uniprottool as uptool
import numpy as np
from modules.rxn.Reaction import Reaction
from modules.rxn.Molecule import Molecule
from tools import ectool as etool
from IPython.display import display_markdown, HTML, SVG
from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize(progress_bar=False)
from tkinter import _flatten
import json
import pandas as pd

FIRST_TIME_RUN = False # For the initial run, please set this flag to True. This will allow the program to download data from UniProt and RHEA, which may take longer depending on your internet speed.

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 1. 读取原始文件

In [2]:
ds_rxn = pd.read_feather(cfg.FILE_DS_RHEA_REACTIONS)
ds_rxn.head(2)

,reaction_id,equation,chebi_id,ec_number,equation_chebi,equation_smiles,equation_chebi_balanced
0,RHEA:22636,dCTP + H2O = dCMP + diphosphate + H(+),CHEBI:61481;CHEBI:15377;CHEBI:57566;CHEBI:3301...,EC:3.6.1.9;EC:3.6.1.12;EC:3.6.1.65,CHEBI:61481 + CHEBI:15377 = CHEBI:57566 + CHEB...,Nc1ccn([C@H]2C[C@H](O)[C@@H](COP([O-])(=O)OP([...,CHEBI:61481 + CHEBI:15377 = CHEBI:57566 + CHEB...
1,RHEA:22640,NADP(+) + sphinganine = 3-oxosphinganine + H(+...,CHEBI:58349;CHEBI:57817;CHEBI:58299;CHEBI:1537...,EC:1.1.1.102,CHEBI:58349 + CHEBI:57817 = CHEBI:58299 + CHEB...,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,CHEBI:58349 + CHEBI:57817 = CHEBI:58299 + CHEB...


## 2.  配平CHEBI 反应方程式

In [61]:
def balance_rhea_chebi_equation(rxn_id, rxn_smiles, rxn_equation, rxn_equation_ref_chebi, rxn_ec):
    rxn = Reaction( rxn_id = rxn_id,
                    rxn_smiles = rxn_smiles,
                    rxn_equation = rxn_equation,
                    rxn_equation_ref_chebi = rxn_equation_ref_chebi,
                    rxn_ec = rxn_ec)
    res =rxn.get_balanced_equation(res_type='ref_chebi')
    
    return res
    

In [63]:
ds_rxn['equation_chebi']= ds_rxn.apply(lambda x:balance_rhea_chebi_equation(rxn_id=x.reaction_id, 
                                                           rxn_smiles=x.equation_smiles,
                                                           rxn_equation = x.equation,
                                                           rxn_equation_ref_chebi=x.equation_chebi,
                                                           rxn_ec=x.ec_number
                                                           ), axis=1)

In [64]:
ds_rxn.head(3)

,reaction_id,equation,chebi_id,ec_number,equation_chebi,equation_smiles,equation_chebi_balanced
0,RHEA:22636,dCTP + H2O = dCMP + diphosphate + H(+),CHEBI:61481;CHEBI:15377;CHEBI:57566;CHEBI:3301...,EC:3.6.1.9;EC:3.6.1.12;EC:3.6.1.65,CHEBI:61481 + CHEBI:15377 = CHEBI:57566 + CHEB...,Nc1ccn([C@H]2C[C@H](O)[C@@H](COP([O-])(=O)OP([...,CHEBI:61481 + CHEBI:15377 = CHEBI:57566 + CHEB...
1,RHEA:22640,NADP(+) + sphinganine = 3-oxosphinganine + H(+...,CHEBI:58349;CHEBI:57817;CHEBI:58299;CHEBI:1537...,EC:1.1.1.102,CHEBI:58349 + CHEBI:57817 = CHEBI:58299 + CHEB...,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP([O-])(=O)...,CHEBI:58349 + CHEBI:57817 = CHEBI:58299 + CHEB...
2,RHEA:22644,O2 + protopine + reduced [NADPH--hemoprotein r...,CHEBI:15379;CHEBI:16415;CHEBI:57618;CHEBI:1710...,EC:1.14.14.98,CHEBI:15379 + CHEBI:16415 + CHEBI:57618 = CHEB...,O=O.CN1CCc2cc3OCOc3cc2C(=O)Cc2ccc3OCOc3c2C1.Cc...,CHEBI:15379 + CHEBI:16415 + CHEBI:57618 = CHEB...


In [69]:
ds_rxn.to_feather(cfg.FILE_DS_RHEA_REACTIONS)